In [1]:
#%pip install -r requirements.txt

import pandas as pd
import numpy as np

In [2]:
#interest rate

df = pd.read_csv('./data/before-convert/interest_rate.csv')
df['year'] = pd.DatetimeIndex(df['TIME']).year
df['month'] = pd.DatetimeIndex(df['TIME']).month

df = df.drop('TIME', axis=1)

df = df.rename(columns = {'LOCATION':'location'})
df = df.drop('Country', axis=1)

df.loc[df['location'] == 'GBR' , 'location'] = 'EU'

longTerm = df.query('Subject == "Long-term interest rates, Per cent per annum"')
longTerm = longTerm.rename(columns = {'Interest-Rate':'interest_rate_long_term'})
longTerm = longTerm.drop('Subject', axis=1)

shortTerm = df.query('Subject == "Short-term interest rates, Per cent per annum"')
shortTerm = shortTerm.rename(columns = {'Interest-Rate':'interest_rate_short_term'})
shortTerm = shortTerm.drop('Subject', axis=1)


immediateTerm = df.query('Subject == "Immediate interest rates, Call Money, Interbank Rate, Per cent per annum"')
immediateTerm = immediateTerm.rename(columns = {'Interest-Rate':'interest_rate'})
immediateTerm = immediateTerm.drop('Subject', axis=1)

#df = pd.merge(longTerm, shortTerm, on=['location', 'year', 'month'] , how='inner', suffixes=('', '_delme'))
#df = pd.merge(df, immediateTerm, on=['location', 'year', 'month'] , how='inner', suffixes=('', '_delme'))
df = immediateTerm
df = df[[c for c in df.columns if not c.endswith('_delme')]]

df.to_csv('./data/interest_rate.csv', index=False)

In [3]:
#cpi

#au
df_au = pd.read_csv('./data/before-convert/cpi_au.csv')

df_au['Date'] = pd.DatetimeIndex(df_au['TIME'])
df_au = df_au.drop('TIME', axis=1)

df_au['Date'] = pd.to_datetime(df_au['Date']).dt.to_period('M')
df_au = df_au.set_index('Date').resample('M').mean()
df_au['location'] = 'AUS'
df_au['year'] = df_au.index.year
df_au['month'] = df_au.index.month

df_au.reset_index(drop=True, inplace=True)


#us
df_us = pd.read_csv('./data/before-convert/cpi_us.csv')

df_final_us = pd.DataFrame(columns=['year','month'])

for index, row in df_us.iterrows():
    for i in range(12):
        df_final_us = pd.concat([df_final_us, pd.DataFrame.from_records([{'year':row[0],'month':i+1, 'cpi-index':row[i+1]}])])

df_final_us['location'] = 'USA'
df_final_us = df_final_us.dropna()
df_final_us['year'] = df_final_us['year'].astype(int)


#eu

df_eu = pd.read_csv('./data/before-convert/cpi_eu.csv')

df_eu['year'] = pd.DatetimeIndex(df_eu['Date'], dayfirst=True).year
df_eu['month'] = pd.DatetimeIndex(df_eu['Date'], dayfirst=True).month
df_eu = df_eu.drop('Date', axis=1)

df = pd.concat([df_au, df_final_us,df_eu])


df.to_csv('./data/cpi.csv', index=False)

In [4]:
#oil_price
lagged_months = [1,2,3,4,5,6]

#au

df_au = pd.read_csv('./data/before-convert/oil_price_au.csv')

df_au['Date'] = pd.to_datetime(df_au['Date'],dayfirst=True)
df_au = df_au.set_index('Date').resample('M').mean()

df_au['year'] = df_au.index.year
df_au['month'] = df_au.index.month
df_au['location'] = 'AUS'

df_au = df_au.rename(columns = {'Value':'oil_price'})
for index, val in enumerate(lagged_months):
    df_au['oil_price_lag_%sm'%(val)] = df_au['oil_price'].shift(freq= '%sM'%(val))

df_au.reset_index(drop=True, inplace=True)

#us

df_us = pd.read_csv('./data/before-convert/oil_price_us.csv')

df_us['year'] = pd.DatetimeIndex(df_us['date'], dayfirst=True).year
df_us['month'] = pd.DatetimeIndex(df_us['date'], dayfirst=True).month
df_us['location'] = 'USA'
df_us = df_us.rename(columns = {'value':'oil_price'})

for index, val in enumerate(lagged_months):
    df_us['oil_price_lag_%sm'%(val)] = df_us['oil_price'].shift(val)

df_us = df_us.drop('date', axis=1)


#eu

df_eu = pd.read_csv('./data/before-convert/oil_price_eu.csv')
df_eu['year'] = pd.DatetimeIndex(df_eu['Date'], dayfirst=True).year
df_eu['month'] = pd.DatetimeIndex(df_eu['Date'], dayfirst=True).month
df_eu['location'] = 'EU'
df_eu = df_eu.rename(columns = {'Median':'oil_price'})

df_eu = df_eu.drop('Date', axis=1)
df_eu = df_eu.drop('Open', axis=1)
df_eu = df_eu.drop('High', axis=1)
df_eu = df_eu.drop('Low', axis=1)
df_eu = df_eu.drop('Close', axis=1)

for index, val in enumerate(lagged_months):
    df_eu['oil_price_lag_%sm'%(val)] = df_eu['oil_price'].shift(val)

df = pd.concat([df_au, df_us, df_eu])

df = df.dropna()

df.to_csv('./data/oil_price.csv', index=False)

In [9]:
#inflation
lagged_months = [9,12]

#au

df_au = pd.read_csv('./data/before-convert/inflation_au.csv')

df_au['DateTime'] = pd.to_datetime(df_au['DateTime'],dayfirst=True).dt.to_period('M')
df_au = df_au.set_index('DateTime').resample('M').interpolate()

df_au['year'] = df_au.index.year
df_au['month'] = df_au.index.month
df_au['location'] = 'AUS'
df_au.reset_index(drop=True, inplace=True)
df_au = df_au.rename(columns = {'Value':'inflation'})

for index, lagged_month in enumerate(lagged_months):                                          
    df_au['inflation_lag_%sm'%(val)] = df_au['inflation'].shift(lagged_month)

#us

df_us = pd.read_csv('./data/before-convert/inflation_us.csv')
df_us['year'] = pd.DatetimeIndex(df_us['DateTime'], dayfirst=True).year
df_us['month'] = pd.DatetimeIndex(df_us['DateTime'], dayfirst=True).month
df_us['location'] = 'USA'
df_us = df_us.drop('DateTime', axis=1)
df_us = df_us.rename(columns = {'Value':'inflation'})

for index, lagged_month in enumerate(lagged_months):                                       
    df_us['inflation_lag_%sm'%(val)] = df_us['inflation'].shift(lagged_month)

#eu
df_eu = pd.read_csv('./data/before-convert/inflation_eu.csv')
df_eu['year'] = pd.DatetimeIndex(df_eu['DateTime'], dayfirst=True).year
df_eu['month'] = pd.DatetimeIndex(df_eu['DateTime'], dayfirst=True).month
df_eu['location'] = 'EU'
df_eu = df_eu.drop('DateTime', axis=1)
df_eu = df_eu.rename(columns = {'Value':'inflation'})

for index, lagged_month in enumerate(lagged_months):                                          
    df_eu['inflation_lag_%sm'%(val)] = df_eu['inflation'].shift(lagged_month)


df = pd.concat([df_au, df_us, df_eu])
df = df.dropna()
df.to_csv('./data/inflation.csv', index=False)